In [3]:
%matplotlib widget
import sys
import os
from pathlib import Path

# Setting up the working directory
def find_project_dir(current_dir):
    for parent in current_dir.parents:
        if (parent / 'README.txt').exists():
            return parent
    return current_dir

current_directory = Path(os.getcwd())
project_dir = find_project_dir(current_directory)
os.chdir(project_dir)

# Creating working directories
notebooks_dir = project_dir / 'BACKGROUND_PYTHON_SCRIPTS'
data_dir = project_dir / 'input'
output_dir = project_dir / 'output'
functions_dir = notebooks_dir / 'FUNCTIONS'
sys.path.append(str(functions_dir))

In [4]:
# Import the packages
import numpy as np
from ipywidgets import interactive, FloatText, FloatSlider, VBox, widgets, Button
from IPython.display import display

# Import the functions
from Import_functions import load_data, clean_data

In [19]:
# Global dictionary to store point information and data
data_store = {'data': None, 'point_name' : None}

# Create input fields for file, point, and stick length
file_name_widget = widgets.Text(
    value='',
    placeholder='Enter the file name of cross profile .pos',
    description='Cross profile file name:',
    disabled=False,
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'}
)

wl_file_name_widget = widgets.Text(
    value='',
    placeholder='Enter the file name of waterlevel .pos',
    description='Waterlevel file name:',
    disabled=False,
    layout=widgets.Layout(width='600px'), 
    style={'description_width': '150px'}
)

vp_file_name_widget = widgets.Text(
    value='',
    placeholder='Enter the file name of velocity profile .csv',
    description='Velocity profilefile name:',
    disabled=False,
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'}                      
)

point_name_widget = widgets.Text(
    value='',
    placeholder='Enter the point name',
    description='Point name:',
    disabled=False,
    layout=widgets.Layout(width='600px'),
    style={'description_width': '150px'}
)

stick_height_widget = widgets.Text(
    value='',
    placeholder='Enter the stick length in [m]',
    description='Sticklength:',
    disabled=False,
    layout=widgets.Layout(width='600px'), 
    style={'description_width': '150px'}
)

# Create the button and output
button = widgets.Button(description="Import data")
output = widgets.Output()

# Create a button click function that uses the load_data and clean_data function
def on_button_click(b):
    with output:
        output.clear_output()
        try:
            file_name = file_name_widget.value.strip()  # remove spaces in input
            wl_file_name = wl_file_name_widget.value.strip()
            vp_file_name = vp_file_name_widget.value.strip()
            point_name = point_name_widget.value.strip()
            stick_length = stick_height_widget.value.strip()  # Correctly access the value attribute

            # Step 1: File validation checks
            if not file_name:
                print("Please enter a .pos file located in the GNSS_DATA_FOLDER.")
                return
            elif not file_name.endswith('.pos'):
                print("Invalid file type. Please enter a file with a .pos extension.")
                return
            
            # Construct the file path (assuming 'data_dir' is defined)
            file_path = data_dir / file_name
            wl_file_path = data_dir / wl_file_name
            # vp_file_path = data_dir / vp_file_name

            # Check if the file exists
            if not file_path.exists() or not file_path.is_file():
                print(f"File '{file_name}' does not exist in the input folder. Please check the file name or location of the file.")
                return
            
            if not wl_file_path.exists() or not wl_file_path.is_file():
                print(f"File '{wl_file_name}' does not exist in the input folder. Please check the file name or location of the file.")
                return
            
            # if not vp_file_path.exists() or not vp_file_path.is_file():
            #     print(f"File '{vp_file_name}' does not exist in the input folder. Please check the file name or location of the file.")
            #     return

            # Step 2: Check if point name is provided
            if not point_name:
                print("Please enter a point name.")
                return

            # Save the point name to the data_store
            data_store['point_name'] = point_name

            # Step 3: Check if stick length is provided
            if not stick_length:
                print("Please enter the stick length.")
                return
            
            # Convert stick length to float
            try:
                stick_length = float(stick_length)
            except ValueError:
                print("Invalid stick length. Please enter a numeric value and use a point as decimal point.")
                return
          
            # Step 4: If all validations pass, proceed to load and display the data
            print(f"File '{file_name}' found! The DataFrame for point '{point_name}' is displayed below:")
            # print(f"File '{wl_file_name}' found! The DataFrame for point '{point_name}' is displayed below:")

            # Use the functions to load and clean the data
            data = load_data(file_path)
            data_cleaned = clean_data(data)
            
            wl_data = load_data(wl_file_path)
            wl_data_cleaned = clean_data(wl_data)
            
            # vp_data = pd.read_csv(vp_file_path, delimiter=';', index_col=0)
            # vp_data = vp_data.replace(',', '.', regex=True).astype(float)

            # Calculate actual height and add the stick length columns
            data_cleaned['sticklength'] = stick_length
            data_cleaned['actual_height(m)'] = data_cleaned['height(m)'] - stick_length
            
            wl_data_cleaned['sticklength'] = stick_length
            wl_data_cleaned['actual_height(m)'] = wl_data_cleaned['height(m)'] - stick_length

            # Display the point name and the cleaned DataFrame
            print(f"Dataframe for: '{point_name}'")
            display(data_cleaned)

            #Save the data to the data_store dataframe
            data_store['data'] = data_cleaned
            data_store['point_name'] = point_name
            data_store['wl_data']  = wl_data_cleaned
            data_store['vp_file_name'] = vp_file_name
            # data_store['vp_data'] = vp_data

        except Exception as e:
            print(f"An error occurred: {e}")

# Attach the button click handler
button.on_click(on_button_click)

# Display the input fields, button, and output in the notebook
display(file_name_widget)
display(wl_file_name_widget)
display(vp_file_name_widget)
display(point_name_widget)
display(stick_height_widget)
display(button)
display(output)

Text(value='', description='Cross profile file name:', layout=Layout(width='600px'), placeholder='Enter the fi…

Text(value='', description='Waterlevel file name:', layout=Layout(width='600px'), placeholder='Enter the file …

Text(value='', description='Velocity profilefile name:', layout=Layout(width='600px'), placeholder='Enter the …

Text(value='', description='Point name:', layout=Layout(width='600px'), placeholder='Enter the point name', st…

Text(value='', description='Sticklength:', layout=Layout(width='600px'), placeholder='Enter the stick length i…

Button(description='Import data', style=ButtonStyle())

Output()